# Generate Test Data

In [1]:
import xarray as xr
import numpy as np
import dask
import intake
from getpass import getuser
from pathlib import Path

import marEx
import marEx.helper as hpc

In [2]:
# Lustre Scratch Directory
scratch_dir = Path('/scratch') / getuser()[0] / getuser()
save_dir = Path('/home') / getuser()[0] / getuser() / 'opt' / 'marEx' / 'tests' / 'data'

In [3]:
# Start Dask Cluster
client = hpc.start_local_cluster(n_workers=32, threads_per_worker=1,
                                 scratch_dir = scratch_dir / 'clients')  # Specify temporary scratch directory for dask to use

Dask Scratch: '/scratch/b/b382615/clients/tmpydvm_1d5'
Memory per Worker: 15.74 GB
Hostname: l40201
Forward Port: l40201:8787
Dashboard Link: localhost:8787/status


# Raw SST Data

In [4]:
cat = intake.open_catalog("https://raw.githubusercontent.com/eerie-project/intake_catalogues/main/eerie.yaml")

In [5]:
# Import 40 years of OSTIA Observations
dat_regrid = cat['dkrz.disk.observations.OSTIA.OSTIA.daily']
sst_regrid = dat_regrid(chunks={}).to_dask().sst.astype(np.float32).coarsen(lat=5,lon=5).mean().rename('to')
sst_regrid

<xarray.DataArray 'to' (time: 14761, lat: 720, lon: 1440)> Size: 61GB
dask.array<mean_agg-aggregate, shape=(14761, 720, 1440), dtype=float32, chunksize=(10, 240, 360), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float32 3kB -89.88 -89.62 -89.38 -89.12 ... 89.38 89.62 89.88
  * lon      (lon) float32 6kB -179.9 -179.6 -179.4 -179.1 ... 179.4 179.6 179.9
  * time     (time) datetime64[ns] 118kB 1982-01-01T12:00:00 ... 2022-05-31T1...
Attributes:
    comment:         OSTIA foundation SST
    long_name:      analysed sea surface temperature
    reference:      C.J. Donlon, M. Martin,J.D. Stark, J. Roberts-Jones, E. F...
    source:         AMSR2-REMSS-L2P-v2.0, AMSRE-REMSS-L2P-v2.0, TMI-REMSS-L2P...
    standard_name:  sea_surface_foundation_temperature
    units:          kelvin
    valid_max:      4500
    valid_min:      -300

In [6]:
# Import 40 years of Daily ICON data (ref. EERIE project)
dat_native = cat['dkrz.disk.model-output.icon-esm-er.eerie-control-1950.v20240618']['ocean']['native']
sst_native = dat_native['2d_daily_mean'](chunks={}).to_dask().to.isel(depth=0).drop_vars({'depth','cell_sea_land_mask'}).chunk({'time':32})
sst_native

<xarray.DataArray 'to' (time: 18262, ncells: 14886338)> Size: 1TB
dask.array<rechunk-merge, shape=(18262, 14886338), dtype=float32, chunksize=(32, 14886338), chunktype=numpy.ndarray>
Coordinates:
    lat      (ncells) float64 119MB dask.array<chunksize=(14886338,), meta=np.ndarray>
    lon      (ncells) float64 119MB dask.array<chunksize=(14886338,), meta=np.ndarray>
  * time     (time) datetime64[ns] 146kB 1991-01-01T23:59:59 ... 2040-12-30T2...
Dimensions without coordinates: ncells
Attributes:
    CDI_grid_type:                unstructured
    long_name:                    sea water potential temperature
    number_of_grid_in_reference:  1
    param:                        18.4.10
    standard_name:                sea_water_potential_temperature
    units:                        C

In [7]:
# Define subset
time_slice_ostia = slice('1982-01-01', '2022-01-01')
time_slice_icon = slice('1991-01-01', '2031-01-01')
regrid_slice_ostia = dict(lat=slice(35, 40), lon=slice(-40, -30))
native_slice_icon = slice(0,1000)

In [8]:
sst_regrid_slice = sst_regrid.sel(time=time_slice_ostia, **regrid_slice_ostia).chunk({'time':30, 'lat':-1,'lon':-1})
sst_regrid_slice

<xarray.DataArray 'to' (time: 14611, lat: 20, lon: 40)> Size: 47MB
dask.array<rechunk-merge, shape=(14611, 20, 40), dtype=float32, chunksize=(30, 20, 40), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float32 80B 35.12 35.38 35.62 35.88 ... 39.38 39.62 39.88
  * lon      (lon) float32 160B -39.88 -39.62 -39.38 ... -30.62 -30.38 -30.12
  * time     (time) datetime64[ns] 117kB 1982-01-01T12:00:00 ... 2022-01-01T1...
Attributes:
    comment:         OSTIA foundation SST
    long_name:      analysed sea surface temperature
    reference:      C.J. Donlon, M. Martin,J.D. Stark, J. Roberts-Jones, E. F...
    source:         AMSR2-REMSS-L2P-v2.0, AMSRE-REMSS-L2P-v2.0, TMI-REMSS-L2P...
    standard_name:  sea_surface_foundation_temperature
    units:          kelvin
    valid_max:      4500
    valid_min:      -300

In [9]:
sst_native_slice = sst_native.sel(time=time_slice_icon).isel(ncells=native_slice_icon).drop_vars({'lat','lon'})
sst_native_slice

<xarray.DataArray 'to' (time: 14611, ncells: 1000)> Size: 58MB
dask.array<getitem, shape=(14611, 1000), dtype=float32, chunksize=(32, 1000), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 117kB 1991-01-01T23:59:59 ... 2031-01-01T2...
Dimensions without coordinates: ncells
Attributes:
    CDI_grid_type:                unstructured
    long_name:                    sea water potential temperature
    number_of_grid_in_reference:  1
    param:                        18.4.10
    standard_name:                sea_water_potential_temperature
    units:                        C

In [10]:
# Save to Zarr
sst_regrid_slice.to_zarr(save_dir / 'sst_gridded.zarr', mode='w')
sst_native_slice.to_zarr(save_dir / 'sst_unstructured.zarr', mode='w')

# Pre-processed Data

In [4]:
time_slice = slice('2010-01-01', '2010-02-01')
extremes_ds = xr.open_zarr(scratch_dir / 'mhws' / 'extremes_binary_gridded_shifting_hobday.zarr', chunks={}).sel(time=time_slice).drop_vars({'thresholds','dat_anomaly','dayofyear'})
extremes_ds

<xarray.Dataset> Size: 34MB
Dimensions:         (time: 32, lat: 720, lon: 1440)
Coordinates:
  * lat             (lat) float32 3kB -89.88 -89.62 -89.38 ... 89.38 89.62 89.88
  * lon             (lon) float32 6kB -179.9 -179.6 -179.4 ... 179.4 179.6 179.9
  * time            (time) datetime64[ns] 256B 2010-01-01T12:00:00 ... 2010-0...
Data variables:
    extreme_events  (time, lat, lon) bool 33MB dask.array<chunksize=(23, 720, 1440), meta=np.ndarray>
    mask            (lat, lon) bool 1MB dask.array<chunksize=(720, 1440), meta=np.ndarray>
Attributes:
    method_anomaly:        shifting_baseline
    method_extreme:        hobday_extreme
    preprocessing_steps:   ['Rolling climatology using 15 years', 'Smoothed w...
    smooth_days_baseline:  21
    threshold_percentile:  95
    window_days_hobday:    11
    window_year_baseline:  15

In [5]:
# Coarsen extremes_ds (lat & lon)
extremes_ds_coarsen = extremes_ds.coarsen(lat=4, lon=4, boundary='trim').any().chunk({'time': 2, 'lat':-1, 'lon':-1}).persist()
extremes_ds_coarsen

<xarray.Dataset> Size: 2MB
Dimensions:         (time: 32, lat: 180, lon: 360)
Coordinates:
  * lat             (lat) float32 720B -89.5 -88.5 -87.5 ... 87.5 88.5 89.5
  * lon             (lon) float32 1kB -179.5 -178.5 -177.5 ... 177.5 178.5 179.5
  * time            (time) datetime64[ns] 256B 2010-01-01T12:00:00 ... 2010-0...
Data variables:
    extreme_events  (time, lat, lon) bool 2MB dask.array<chunksize=(2, 180, 360), meta=np.ndarray>
    mask            (lat, lon) bool 65kB dask.array<chunksize=(180, 360), meta=np.ndarray>
Attributes:
    method_anomaly:        shifting_baseline
    method_extreme:        hobday_extreme
    preprocessing_steps:   ['Rolling climatology using 15 years', 'Smoothed w...
    smooth_days_baseline:  21
    threshold_percentile:  95
    window_days_hobday:    11
    window_year_baseline:  15

In [6]:
# Clear encoding so we can write to Zarr
extremes_ds_coarsen.encoding = {}
for var in extremes_ds_coarsen.data_vars:
	extremes_ds_coarsen[var].encoding = {}

In [7]:
extremes_ds_coarsen.to_zarr(save_dir / 'extremes_gridded.zarr', mode='w')